# config

> This module provides the core config class for qgofer

In [ ]:
# | default_exp config

In [ ]:
# | hide
import nbdev
# | hide
from nbdev.showdoc import *

In [ ]:
!pip install black

In [ ]:
# | export
from __future__ import annotations

import logging
import shutil
import sys
from pathlib import Path

In [ ]:
# | hide
# The Integrity error is expected it's not a bug!
from sqlite3 import IntegrityError
from typing import Any, Dict

In [ ]:
# | hide
%run 00_core.ipynb
%run 01_db.ipynb
%run 02_logger.ipynb

sys.path.append("..")

In [ ]:
# | export
try:
    from qgoferutils.db import QGoferDBWrapper
    from qgoferutils.logger import get_log_path, get_logger
except:
    pass

In [ ]:
# | export


class QGoferConfig:
    """The main appliation class for managing the entirety of qgofer.

    Attributes:
        home: The home directory of the user, defaults to Path.home()
        root_dir: The root directory to index, defaults to Path.home()
    """

    __slots__ = (
        "_home",
        "_root_dir",
        "_qgofer",
        "_qgofer_config",
        "_qgofer_cache",
        "_qgofer_cache_db",
        "_qgofer_logs",
        "_log_path",
        "_logger",
    )
    _instances = {}

    def __new__(cls, home: Path = Path.home(), root_dir: Path = Path.home()):
        """Create a new instance of the class. by using the singleton pattern."""
        if cls not in cls._instances:
            cls._instances[cls] = super().__new__(cls)
        return cls._instances[cls]

    def __init__(self, home: Path = Path.home(), root_dir: Path = Path.home()):
        """Initialize the class. using home and root_dir as the home and root directory of qgofer."""
        self._home = home
        self._root_dir = root_dir
        self._qgofer = self._home / ".qgofer"
        self._qgofer.mkdir(parents=True, exist_ok=True)
        self._qgofer_config = self._qgofer / "config.toml"
        self._qgofer_config.touch(exist_ok=True)
        self._qgofer_cache = self._qgofer / "cache"
        self._qgofer_cache.mkdir(parents=True, exist_ok=True)
        self._qgofer_cache_db = self._qgofer_cache / "qgofer.db"
        self._qgofer_cache_db.touch(exist_ok=True)
        self._qgofer_logs = self._qgofer / "logs"
        self._qgofer_logs.mkdir(parents=True, exist_ok=True)
        self._log_path = get_log_path()
        self._logger = get_logger(log_path=self._log_path)

    @property
    def home(self) -> Path:
        """Return the home directory of the user."""
        return self._home

    @property
    def qgofer(self) -> Path:
        """Return the qgofer directory."""
        return self._qgofer

    @property
    def qgofer_config(self) -> Path:
        """Return the qgofer config file."""
        return self._qgofer_config

    @property
    def qgofer_cache(self) -> Path:
        """Return the qgofer cache directory."""
        return self._qgofer_cache

    @property
    def qgofer_cache_db(self) -> Path:
        """Return the qgofer cache database."""
        return self._qgofer_cache_db

    @property
    def logger(self) -> logging.Logger:
        """Return the logger for QGoferConfig"""
        return self._logger

    @property
    def logger_path(self) -> Path:
        """Return logger path for QGoferConfig"""
        return self._log_path

    @property
    def root_dir(self) -> Path:
        """Return the root directory of the user"""
        return self._root_dir

    def _clear_cache_db(self) -> None:
        """Clear the qgofer cache database."""
        qgofer_db_wrapper = QGoferDBWrapper(self.qgofer_cache_db)
        try:
            os.remove(self.qgofer_cache_db)
        except FileNotFoundError as err:
            self._logger.error(err)

    def clear_cache(self) -> None:
        """Clears the cache folder"""
        try:
            shutil.rmtree(self._qgofer_cache)
        except OSError as err:
            self._logger.error(err)
        self._clear_cache_db()

    def clear_logs(self) -> None:
        """This function clears the logs folder"""
        try:
            shutil.rmtree(self._qgofer_logs)
            self._qgofer_logs.mkdir(parents=True, exist_ok=True)
        except OSError as err:
            self._logger.error(err)

    def __repr__(self) -> str:
        """Return the representation of the class."""
        return f"{self.__class__.__name__}({self.home})"

    def __str__(self) -> str:
        """Return the string representation of the class."""
        return "Qgofer your personal assistant"

In [ ]:
# | export


def create_config_table(qgofer_cache_db) -> None:
    """Create the qgofer config table."""
    qgofer_db_wrapper = QGoferDBWrapper(qgofer_cache_db)
    qgofer_db_wrapper.create_table("qgofer_config", "key TEXT UNIQUE,value TEXT")

In [ ]:
# | export


def store_hugging_face_token(API_TOKEN, qgofer_cache_db) -> None:
    """store the hugging face api token"""
    qgofer_db_wrapper = QGoferDBWrapper(qgofer_cache_db)
    qgofer_db_wrapper.insert(
        "qgofer_config",
        columns="key, value",
        values=f"'HUGGING_FACE_KEY', '{API_TOKEN}'",
    )

In [ ]:
q = QGoferConfig()
create_config_table(q.qgofer_cache_db)
try:
    store_hugging_face_token("what a wow", q.qgofer_cache_db)
except IntegrityError as e:
    print("IntegrityError occurred:", e)

2024-03-07 21:36:20,578 - qgofer - INFO - Logging to C:\Users\acqua\.qgofer\logs/tOk1ii33REygYB2zGpzuaw_log_24_03_07_21_36_20.log
IntegrityError occurred: UNIQUE constraint failed: qgofer_config.key


In [ ]:
#| hide
nbdev.nbdev_export()